In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from copy import deepcopy

In [2]:
path = 'Data/'

In [3]:
def load_csv(path, csvf):
    df = pd.read_csv(path + csvf)
    print("Shape of loaded df: ", df.shape)
#    print(df.head())
    return df

In [4]:
def merge_with_target(df):
    df_train = df[df['SK_ID_CURR'].isin(id_to_target['SK_ID_CURR'])]
    df_train = df_train.merge(id_to_target, on='SK_ID_CURR', how='inner')
    print("The shape of the df merged with the target is: ", df_train.shape)
    return df_train

In [5]:
def find_corrs(df, col=None):
    if col == None:
        correlations = df.corr()['TARGET'].sort_values(ascending=False)
        if len(correlations) > 30:
            print('Most Positive Correlations:\n', correlations.head(15))
            print('\nMost Negative Correlations:\n', correlations.tail(15))
        else:
            print('Correlations, from highest to lowest:\n', correlations)
        return correlations
    else:
        correlation = df[col].corr(df['TARGET'])
        print('The correlation of', col, 'with the target is', str(correlation))
        return correlation

In [6]:
'''Applies label encoding to columns with only 2 unique categories'''
def label_df(df):
    # Create a label encoder object
    le = LabelEncoder()
    le_count = 0

    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 2:
                # Train on the training data
                le.fit(df[col])
                # Transform both training and testing data
                df[col] = le.transform(df[col])

                # Keep track of how many columns were label encoded
                le_count += 1

    print('%d columns were label encoded.' % le_count)

In [7]:
app_train = load_csv(path, 'app_train_smooth.csv')

Shape of loaded df:  (307511, 241)


In [8]:
app_test = load_csv(path, 'app_test_smooth.csv')

Shape of loaded df:  (48744, 240)


In [9]:
id_to_target = app_train[['SK_ID_CURR', 'TARGET']]

### Checking Correlation With Previous Application

In [40]:
prev_app = load_csv(path, 'previous_application.csv')
prev_app.head()

Shape of loaded df:  (1670214, 37)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
prev_app_train = merge_with_target(prev_app)
prev_app_train.head()


The shape of the df merged with the target is:  (1413701, 38)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,TARGET
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0,0
1,1696966,271877,Consumer loans,68258.655,1800000.0,1754721.0,180000.0,1800000.0,SATURDAY,18,...,36.0,low_normal,POS industry with interest,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2154916,271877,Consumer loans,12417.390,108400.5,119848.5,0.0,108400.5,SUNDAY,14,...,12.0,middle,POS industry with interest,365243.0,-512.0,-182.0,-392.0,-387.0,0.0,0
3,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0,0
4,1536272,108129,Cash loans,21709.125,450000.0,512370.0,NaN,450000.0,WEDNESDAY,9,...,36.0,low_normal,Cash X-Sell: low,365243.0,-485.0,565.0,-155.0,-147.0,1.0,0


In [42]:
label_df(prev_app_train)

1 columns were label encoded.


In [43]:
# one-hot encoding of categorical variables
prev_app_train = pd.get_dummies(prev_app_train)

print('Training Features shape: ', prev_app_train.shape)

Training Features shape:  (1413701, 163)


In [14]:
corrs = find_corrs(prev_app_train)

Most Positive Correlations:
 TARGET                             1.000000
NAME_CONTRACT_STATUS_Refused       0.054458
NAME_PRODUCT_TYPE_walk-in          0.042842
DAYS_DECISION                      0.039901
CODE_REJECT_REASON_SCOFR           0.038432
CODE_REJECT_REASON_HC              0.037158
CNT_PAYMENT                        0.030480
NAME_GOODS_CATEGORY_XNA            0.029541
RATE_INTEREST_PRIVILEGED           0.028640
NAME_YIELD_GROUP_XNA               0.028366
CHANNEL_TYPE_AP+ (Cash loan)       0.028294
NAME_PAYMENT_TYPE_XNA              0.026353
CODE_REJECT_REASON_LIMIT           0.025967
NAME_SELLER_INDUSTRY_XNA           0.025203
PRODUCT_COMBINATION_Card Street    0.025096
Name: TARGET, dtype: float64

Most Negative Correlations:
 CHANNEL_TYPE_Stone                               -0.017659
NAME_GOODS_CATEGORY_Furniture                    -0.018184
NAME_YIELD_GROUP_low_action                      -0.019016
PRODUCT_COMBINATION_POS industry with interest   -0.020967
PRODUCT_COMBINAT

In [44]:
# NAME_CONTRACT_STATUS_Refused       0.054458
# NAME_PRODUCT_TYPE_walk-in          0.042842
# DAYS_DECISION                      0.039901
# CODE_REJECT_REASON_SCOFR           0.038432
# CODE_REJECT_REASON_HC              0.037158
# CNT_PAYMENT                        0.030480
# NAME_PORTFOLIO_POS                               -0.030791
# DAYS_FIRST_DRAWING                               -0.031154
# NAME_CONTRACT_STATUS_Approved                    -0.049161
# CODE_REJECT_REASON_XAP                           -0.052015

features = (
    ['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_STATUS_Refused', 'NAME_PRODUCT_TYPE_walk-in', 
     'DAYS_DECISION', 'CODE_REJECT_REASON_SCOFR', 'CODE_REJECT_REASON_HC', 'CNT_PAYMENT', 
     'NAME_PORTFOLIO_POS', 'DAYS_FIRST_DRAWING', 'NAME_CONTRACT_STATUS_Approved', 'CODE_REJECT_REASON_XAP'])
features

['SK_ID_PREV',
 'SK_ID_CURR',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_PRODUCT_TYPE_walk-in',
 'DAYS_DECISION',
 'CODE_REJECT_REASON_SCOFR',
 'CODE_REJECT_REASON_HC',
 'CNT_PAYMENT',
 'NAME_PORTFOLIO_POS',
 'DAYS_FIRST_DRAWING',
 'NAME_CONTRACT_STATUS_Approved',
 'CODE_REJECT_REASON_XAP']

In [45]:
prev_app.shape

(1670214, 37)

In [46]:
prev_app_train.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,...,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,1,1,...,0,0,0,0,0,0,1,0,0,0
1,1696966,271877,68258.655,1800000.0,1754721.0,180000.0,1800000.0,18,1,1,...,0,0,0,0,1,0,0,0,0,0
2,2154916,271877,12417.390,108400.5,119848.5,0.0,108400.5,14,1,1,...,0,0,0,0,1,0,0,0,0,0
3,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,1,1,...,1,0,0,0,0,0,0,0,0,0
4,1536272,108129,21709.125,450000.0,512370.0,NaN,450000.0,9,1,1,...,1,0,0,0,0,0,0,0,0,0


In [47]:
label_df(prev_app)
prev_app = pd.get_dummies(prev_app)

1 columns were label encoded.


In [48]:
prev_app.shape

(1670214, 163)

In [49]:
prev_app_trim = prev_app[features]
prev_app_trim.shape

(1670214, 12)

In [51]:
prev_app_trim.describe()
# Cats
# NAME_CONTRACT_STATUS_Refused, NAME_PRODUCT_TYPE_walk-in, CODE_REJECT_REASON_SCOFR, 
# CODE_REJECT_REASON_HC, NAME_PORTFOLIO_POS, NAME_CONTRACT_STATUS_Approved, CODE_REJECT_REASON_XAP

# Conts
# DAYS_DECISION, CNT_PAYMENT, DAYS_FIRST_DRAWING

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_STATUS_Refused,NAME_PRODUCT_TYPE_walk-in,DAYS_DECISION,CODE_REJECT_REASON_SCOFR,CODE_REJECT_REASON_HC,CNT_PAYMENT,NAME_PORTFOLIO_POS,DAYS_FIRST_DRAWING,NAME_CONTRACT_STATUS_Approved,CODE_REJECT_REASON_XAP
count,1.670214e+06,1.670214e+06,1.670214e+06,1.670214e+06,1.670214e+06,1.670214e+06,1.670214e+06,1.297984e+06,1.670214e+06,997149.000000,1.670214e+06,1.670214e+06
mean,1.923089e+06,2.783572e+05,1.740364e-01,8.996512e-02,-8.806797e+02,7.670275e-03,1.049153e-01,1.605408e+01,4.137260e-01,342209.855039,6.207474e-01,8.101315e-01
std,5.325980e+05,1.028148e+05,3.791409e-01,2.861319e-01,7.790997e+02,8.724360e-02,3.064443e-01,1.456729e+01,4.925007e-01,88916.115834,4.852012e-01,3.921971e-01
min,1.000001e+06,1.000010e+05,0.000000e+00,0.000000e+00,-2.922000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2922.000000,0.000000e+00,0.000000e+00
25%,1.461857e+06,1.893290e+05,0.000000e+00,0.000000e+00,-1.300000e+03,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,365243.000000,0.000000e+00,1.000000e+00
50%,1.923110e+06,2.787145e+05,0.000000e+00,0.000000e+00,-5.810000e+02,0.000000e+00,0.000000e+00,1.200000e+01,0.000000e+00,365243.000000,1.000000e+00,1.000000e+00
75%,2.384280e+06,3.675140e+05,0.000000e+00,0.000000e+00,-2.800000e+02,0.000000e+00,0.000000e+00,2.400000e+01,1.000000e+00,365243.000000,1.000000e+00,1.000000e+00
max,2.845382e+06,4.562550e+05,1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,1.000000e+00,8.400000e+01,1.000000e+00,365243.000000,1.000000e+00,1.000000e+00


In [55]:
prev_app_cats = prev_app_trim[['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_STATUS_Refused', 'NAME_PRODUCT_TYPE_walk-in', 
                         'CODE_REJECT_REASON_SCOFR', 'CODE_REJECT_REASON_HC', 'NAME_PORTFOLIO_POS', 
                         'NAME_CONTRACT_STATUS_Approved', 'CODE_REJECT_REASON_XAP']]

prev_app_cnts = prev_app_trim[['SK_ID_PREV', 'SK_ID_CURR', 'DAYS_DECISION', 'CNT_PAYMENT', 
                         'DAYS_FIRST_DRAWING']]

In [59]:
# Group by the client id, calculate aggregation statistics
prev_app_cats_agg = (prev_app_cats
                    .drop(columns = ['SK_ID_PREV'])
                    .groupby('SK_ID_CURR', as_index = False)
                    .agg(['sum'])
                    .reset_index())
prev_app_cats_agg.head()

,SK_ID_CURR,NAME_CONTRACT_STATUS_Refused,NAME_PRODUCT_TYPE_walk-in,CODE_REJECT_REASON_SCOFR,CODE_REJECT_REASON_HC,NAME_PORTFOLIO_POS,NAME_CONTRACT_STATUS_Approved,CODE_REJECT_REASON_XAP
,,sum,sum,sum,sum,sum,sum,sum
0,100001,0,0,0,0,1,1,1
1,100002,0,0,0,0,1,1,1
2,100003,0,0,0,0,2,3,3
3,100004,0,0,0,0,1,1,1
4,100005,0,0,0,0,1,1,2


In [60]:
# Group by the client id, calculate aggregation statistics
prev_app_cnts_agg = (prev_app_cnts
                    .drop(columns = ['SK_ID_PREV'])
                    .groupby('SK_ID_CURR', as_index = False)
                    .agg(['mean'])
                    .reset_index())
prev_app_cnts_agg.head()

,SK_ID_CURR,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING
,,mean,mean,mean
0,100001,-1740.0,8.0,365243.0
1,100002,-606.0,24.0,365243.0
2,100003,-1305.0,10.0,365243.0
3,100004,-815.0,4.0,365243.0
4,100005,-536.0,12.0,365243.0


In [61]:
#Merge with the cats and conts
prev_app_trim = prev_app_cats_agg.merge(prev_app_cnts_agg, on = 'SK_ID_CURR', how = 'inner')
prev_app_trim.head()

/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3108: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,SK_ID_CURR,NAME_CONTRACT_STATUS_Refused,NAME_PRODUCT_TYPE_walk-in,CODE_REJECT_REASON_SCOFR,CODE_REJECT_REASON_HC,NAME_PORTFOLIO_POS,NAME_CONTRACT_STATUS_Approved,CODE_REJECT_REASON_XAP,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING
,,sum,sum,sum,sum,sum,sum,sum,mean,mean,mean
0,100001,0,0,0,0,1,1,1,-1740.0,8.0,365243.0
1,100002,0,0,0,0,1,1,1,-606.0,24.0,365243.0
2,100003,0,0,0,0,2,3,3,-1305.0,10.0,365243.0
3,100004,0,0,0,0,1,1,1,-815.0,4.0,365243.0
4,100005,0,0,0,0,1,1,2,-536.0,12.0,365243.0


In [138]:
print(columns)

['SK_ID_CURR', 'CNT_PAYMENT_', 'CNT_PAYMENT_mean', 'CODE_REJECT_REASON_HC_', 'CODE_REJECT_REASON_HC_mean', 'CODE_REJECT_REASON_SCOFR_', 'CODE_REJECT_REASON_SCOFR_mean', 'CODE_REJECT_REASON_XAP_', 'CODE_REJECT_REASON_XAP_mean', 'DAYS_DECISION_', 'DAYS_DECISION_mean', 'DAYS_FIRST_DRAWING_', 'DAYS_FIRST_DRAWING_mean', 'NAME_CONTRACT_STATUS_Approved_', 'NAME_CONTRACT_STATUS_Approved_mean', 'NAME_CONTRACT_STATUS_Refused_', 'NAME_CONTRACT_STATUS_Refused_mean', 'NAME_PORTFOLIO_POS_', 'NAME_PORTFOLIO_POS_mean', 'NAME_PRODUCT_TYPE_walk-in_', 'NAME_PRODUCT_TYPE_walk-in_mean']


In [140]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in prev_app_trim.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in prev_app_trim.columns.levels[1][1:-1]:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))
            
# Assign the list of columns names as the dataframe column names
prev_app_trim.columns = columns

In [142]:
prev_app_trim.head()

,SK_ID_CURR,CNT_PAYMENT_mean,CODE_REJECT_REASON_HC_mean,CODE_REJECT_REASON_SCOFR_mean,CODE_REJECT_REASON_XAP_mean,DAYS_DECISION_mean,DAYS_FIRST_DRAWING_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_PORTFOLIO_POS_mean,NAME_PRODUCT_TYPE_walk-in_mean
0,100001,0,0,0,0,1,1,1,-1740.0,8.0,365243.0
1,100002,0,0,0,0,1,1,1,-606.0,24.0,365243.0
2,100003,0,0,0,0,2,3,3,-1305.0,10.0,365243.0
3,100004,0,0,0,0,1,1,1,-815.0,4.0,365243.0
4,100005,0,0,0,0,1,1,2,-536.0,12.0,365243.0


In [145]:
'''Write to a csv'''
prev_app_trim.to_csv(path + 'prev_app_trim.csv', index=False)

### Make a function to perform all of the above steps until making the trim DF

In [92]:
def check_corrs(path, file):
    df = load_csv(path, file)
    label_df(df)
    df = pd.get_dummies(df)    
    print('Original shape: ', df.shape)
    
    df_train = merge_with_target(df)
    print('Merged with Target  shape: ', df_train.shape)
    
    corrs = find_corrs(df_train)

    return df, df_train, corrs

### Checking correlation with Bureau.csv

In [63]:
bureau, bureau_train, corrs = check_corrs(path, 'bureau.csv')

Shape of loaded df:  (1716428, 17)
The shape of the df merged with the target is:  (1465325, 18)
0 columns were label encoded.
Training Features shape:  (1465325, 38)
Most Positive Correlations:
 TARGET                                                1.000000
DAYS_CREDIT                                           0.061556
CREDIT_ACTIVE_Active                                  0.042045
DAYS_CREDIT_UPDATE                                    0.041076
DAYS_ENDDATE_FACT                                     0.039057
CREDIT_TYPE_Microloan                                 0.037583
DAYS_CREDIT_ENDDATE                                   0.026497
CREDIT_TYPE_Credit card                               0.020331
AMT_CREDIT_SUM_OVERDUE                                0.006253
CREDIT_ACTIVE_Sold                                    0.005299
CREDIT_TYPE_Loan for working capital replenishment    0.002939
CREDIT_DAY_OVERDUE                                    0.002652
CREDIT_CURRENCY_currency 1                      

In [64]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [65]:
bureau_features = (
    ['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT', 'CREDIT_ACTIVE_Active', 'DAYS_CREDIT_UPDATE',
     'DAYS_ENDDATE_FACT', 'CREDIT_TYPE_Microloan'])

In [68]:
bureau_trim = bureau[bureau_features]
bureau_trim.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_ACTIVE_Active,DAYS_CREDIT_UPDATE,DAYS_ENDDATE_FACT,CREDIT_TYPE_Microloan
0,215354,5714462,-497,0,-131,-153.0,0
1,215354,5714463,-208,1,-20,NaN,0
2,215354,5714464,-203,1,-16,NaN,0
3,215354,5714465,-203,1,-16,NaN,0
4,215354,5714466,-629,1,-21,NaN,0


In [69]:
bureau_trim.describe()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_ACTIVE_Active,DAYS_CREDIT_UPDATE,DAYS_ENDDATE_FACT,CREDIT_TYPE_Microloan
count,1.716428e+06,1.716428e+06,1.716428e+06,1.716428e+06,1.716428e+06,1.082775e+06,1.716428e+06
mean,2.782149e+05,5.924434e+06,-1.142108e+03,3.673950e-01,-5.937483e+02,-1.017437e+03,7.231879e-03
std,1.029386e+05,5.322657e+05,7.951649e+02,4.820955e-01,7.207473e+02,7.140106e+02,8.473242e-02
min,1.000010e+05,5.000000e+06,-2.922000e+03,0.000000e+00,-4.194700e+04,-4.202300e+04,0.000000e+00
25%,1.888668e+05,5.463954e+06,-1.666000e+03,0.000000e+00,-9.080000e+02,-1.489000e+03,0.000000e+00
50%,2.780550e+05,5.926304e+06,-9.870000e+02,0.000000e+00,-3.950000e+02,-8.970000e+02,0.000000e+00
75%,3.674260e+05,6.385681e+06,-4.740000e+02,1.000000e+00,-3.300000e+01,-4.250000e+02,0.000000e+00
max,4.562550e+05,6.843457e+06,0.000000e+00,1.000000e+00,3.720000e+02,0.000000e+00,1.000000e+00


In [73]:
#Cats
#CREDIT_ACTIVE_Active, CREDIT_TYPE_Microloan
#Cnts
#DAYS_CREDIT, DAYS_CREDIT_UPDATE, DAYS_ENDDATE_FACT
bureau_cats = bureau_trim[['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Active',
                           'CREDIT_TYPE_Microloan']]

bureau_cnts = bureau_trim[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT', 'DAYS_CREDIT_UPDATE', 
                           'DAYS_ENDDATE_FACT']]

In [74]:
# Group by the client id, calculate aggregation statistics
bureau_cats_agg = (bureau_cats
                    .drop(columns = ['SK_ID_BUREAU'])
                    .groupby('SK_ID_CURR', as_index = False)
                    .agg(['sum'])
                    .reset_index())
bureau_cats_agg.head()

,SK_ID_CURR,CREDIT_ACTIVE_Active,CREDIT_TYPE_Microloan
,,sum,sum
0,100001,3,0
1,100002,2,0
2,100003,1,0
3,100004,0,0
4,100005,2,0


In [75]:
# Group by the client id, calculate aggregation statistics
bureau_cnts_agg = (bureau_cnts
                    .drop(columns = ['SK_ID_BUREAU'])
                    .groupby('SK_ID_CURR', as_index = False)
                    .agg(['mean'])
                    .reset_index())
bureau_cnts_agg.head()

,SK_ID_CURR,DAYS_CREDIT,DAYS_CREDIT_UPDATE,DAYS_ENDDATE_FACT
,,mean,mean,mean
0,100001,-735.000000,-93.142857,-825.500000
1,100002,-874.000000,-499.875000,-697.500000
2,100003,-1400.750000,-816.000000,-1097.333333
3,100004,-867.000000,-532.000000,-532.500000
4,100005,-190.666667,-54.333333,-123.000000


In [89]:
#Merge with the cats and conts
bureau_trim = bureau_cats_agg.merge(bureau_cnts_agg, on = 'SK_ID_CURR', how = 'inner')
bureau_trim.shape

/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3108: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


(305811, 6)

In [143]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_trim.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_trim.columns.levels[1][1:-1]:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))
            
# Assign the list of columns names as the dataframe column names
bureau_trim.columns = columns

In [144]:
bureau_trim.head()

,SK_ID_CURR,CREDIT_ACTIVE_Active_mean,CREDIT_TYPE_Microloan_mean,DAYS_CREDIT_mean,DAYS_CREDIT_UPDATE_mean,DAYS_ENDDATE_FACT_mean
0,100001,3,0,-735.000000,-93.142857,-825.500000
1,100002,2,0,-874.000000,-499.875000,-697.500000
2,100003,1,0,-1400.750000,-816.000000,-1097.333333
3,100004,0,0,-867.000000,-532.000000,-532.500000
4,100005,2,0,-190.666667,-54.333333,-123.000000


In [146]:
bureau_trim.to_csv(path + 'bureau_trim.csv', index=False)

### Checking correlation with POS_CASH_balance

In [93]:
POS_CASH, POS_CASH_train, corrs = check_corrs(path, 'POS_CASH_balance.csv')

Shape of loaded df:  (10001358, 8)
0 columns were label encoded.
Original shape:  (10001358, 16)
The shape of the df merged with the target is:  (8543375, 17)
Merged with Target  shape:  (8543375, 17)
Correlations, from highest to lowest:
 TARGET                                        1.000000
CNT_INSTALMENT_FUTURE                         0.021972
MONTHS_BALANCE                                0.020147
CNT_INSTALMENT                                0.018506
SK_DPD                                        0.009866
SK_DPD_DEF                                    0.008594
NAME_CONTRACT_STATUS_Amortized debt           0.006732
NAME_CONTRACT_STATUS_Demand                   0.006062
NAME_CONTRACT_STATUS_Returned to the store    0.002596
NAME_CONTRACT_STATUS_Signed                   0.001669
NAME_CONTRACT_STATUS_Canceled                 0.000423
NAME_CONTRACT_STATUS_Completed                0.000412
NAME_CONTRACT_STATUS_Approved                 0.000170
SK_ID_PREV                                   

### Checking correlation with credit_card_balance

In [94]:
credit_card, credit_card_train, corrs = check_corrs(path, 'credit_card_balance.csv')

Shape of loaded df:  (3840312, 23)
0 columns were label encoded.
Original shape:  (3840312, 29)
The shape of the df merged with the target is:  (3227965, 30)
Merged with Target  shape:  (3227965, 30)
Correlations, from highest to lowest:
 TARGET                                1.000000
AMT_BALANCE                           0.050098
AMT_TOTAL_RECEIVABLE                  0.049839
AMT_RECIVABLE                         0.049803
AMT_RECEIVABLE_PRINCIPAL              0.049692
AMT_INST_MIN_REGULARITY               0.039798
CNT_DRAWINGS_ATM_CURRENT              0.038437
CNT_DRAWINGS_CURRENT                  0.037793
MONTHS_BALANCE                        0.035695
CNT_DRAWINGS_POS_CURRENT              0.029536
AMT_DRAWINGS_ATM_CURRENT              0.024700
AMT_DRAWINGS_CURRENT                  0.022378
AMT_CREDIT_LIMIT_ACTUAL               0.013823
AMT_PAYMENT_CURRENT                   0.012929
AMT_PAYMENT_TOTAL_CURRENT             0.012302
SK_DPD_DEF                            0.010538
NAME_CONT

In [95]:
credit_card.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,35.0,0,0,1,0,0,0,0,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,69.0,0,0,1,0,0,0,0,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,30.0,0,0,1,0,0,0,0,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,10.0,0,0,1,0,0,0,0,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,101.0,0,0,1,0,0,0,0,0,0


In [98]:
len(credit_card)

3840312

In [99]:
credit_card_features = (
        ['SK_ID_CURR', 'SK_ID_PREV','AMT_BALANCE', 'AMT_TOTAL_RECEIVABLE', 'AMT_RECIVABLE', 
         'AMT_RECEIVABLE_PRINCIPAL', 'AMT_INST_MIN_REGULARITY', 'CNT_DRAWINGS_ATM_CURRENT', 
         'CNT_DRAWINGS_CURRENT', 'MONTHS_BALANCE', 'CNT_DRAWINGS_POS_CURRENT'])

In [100]:
credit_card_trim = credit_card[credit_card_features]
credit_card_trim.shape

(3840312, 11)

In [101]:
credit_card_trim.describe()

,SK_ID_CURR,SK_ID_PREV,AMT_BALANCE,AMT_TOTAL_RECEIVABLE,AMT_RECIVABLE,AMT_RECEIVABLE_PRINCIPAL,AMT_INST_MIN_REGULARITY,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,MONTHS_BALANCE,CNT_DRAWINGS_POS_CURRENT
count,3.840312e+06,3.840312e+06,3.840312e+06,3.840312e+06,3.840312e+06,3.840312e+06,3.535076e+06,3.090496e+06,3.840312e+06,3.840312e+06,3.090496e+06
mean,2.783242e+05,1.904504e+06,5.830016e+04,5.809829e+04,5.808881e+04,5.596588e+04,3.540204e+03,3.094490e-01,7.031439e-01,-3.452192e+01,5.594791e-01
std,1.027045e+05,5.364695e+05,1.063070e+05,1.059718e+05,1.059654e+05,1.025336e+05,5.600154e+03,1.100401e+00,3.190347e+00,2.666775e+01,3.240649e+00
min,1.000060e+05,1.000018e+06,-4.202502e+05,-4.202502e+05,-4.202502e+05,-4.233058e+05,0.000000e+00,0.000000e+00,0.000000e+00,-9.600000e+01,0.000000e+00
25%,1.895170e+05,1.434385e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.500000e+01,0.000000e+00
50%,2.783960e+05,1.897122e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.800000e+01,0.000000e+00
75%,3.675800e+05,2.369328e+06,8.904669e+04,8.891451e+04,8.889949e+04,8.535924e+04,6.633911e+03,0.000000e+00,0.000000e+00,-1.100000e+01,0.000000e+00
max,4.562500e+05,2.843496e+06,1.505902e+06,1.493338e+06,1.493338e+06,1.472317e+06,2.028820e+05,5.100000e+01,1.650000e+02,-1.000000e+00,1.650000e+02


In [104]:
#Cats
#None
#Cnts
#All

credit_card_cnts = deepcopy(credit_card_trim)

In [109]:
# Group by the client id, calculate aggregation statistics
credit_card_cnts_sum = (credit_card_cnts
                    .drop(columns = ['SK_ID_PREV'])
                    .groupby('SK_ID_CURR', as_index = False)
                    .agg(['sum'])
                    .reset_index())
credit_card_cnts_sum = credit_card_cnts_sum[['SK_ID_CURR', 'AMT_BALANCE', 'MONTHS_BALANCE']]
credit_card_cnts_sum.head()

,SK_ID_CURR,AMT_BALANCE,MONTHS_BALANCE
,,sum,sum
0,100006,0.000,-21
1,100011,4031676.225,-2849
2,100013,1743352.245,-4656
3,100021,0.000,-170
4,100023,0.000,-60


In [119]:
temp = credit_card_cnts_sum['AMT_BALANCE'] / abs(credit_card_cnts_sum['MONTHS_BALANCE'])
credit_card_cnts_sum = credit_card_cnts_sum.assign(AMT_BAL_DIV_MONTHS = temp.values)
credit_card_cnts_sum.head()

,SK_ID_CURR,AMT_BALANCE,MONTHS_BALANCE,AMT_BAL_DIV_MONTHS
,,sum,sum,
0,100006,0.000,-21,0.000000
1,100011,4031676.225,-2849,1415.119770
2,100013,1743352.245,-4656,374.431324
3,100021,0.000,-170,0.000000
4,100023,0.000,-60,0.000000


In [120]:
# Group by the client id, calculate aggregation statistics
credit_card_cnts_agg = (credit_card_cnts
                    .drop(columns = ['SK_ID_PREV'])
                    .groupby('SK_ID_CURR', as_index = False)
                    .agg(['mean'])
                    .reset_index())
credit_card_cnts_agg.head()

,SK_ID_CURR,AMT_BALANCE,AMT_TOTAL_RECEIVABLE,AMT_RECIVABLE,AMT_RECEIVABLE_PRINCIPAL,AMT_INST_MIN_REGULARITY,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,MONTHS_BALANCE,CNT_DRAWINGS_POS_CURRENT
,,mean,mean,mean,mean,mean,mean,mean,mean,mean
0,100006,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-3.5,NaN
1,100011,54482.111149,54433.179122,54433.179122,52402.088919,3956.221849,0.054054,0.054054,-38.5,0.0
2,100013,18159.919219,18101.079844,18101.079844,17255.559844,1454.539551,0.255556,0.239583,-48.5,0.0
3,100021,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-10.0,NaN
4,100023,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-7.5,NaN


In [164]:
#Merge with the means and div_months
credit_card_trim = credit_card_cnts_agg.merge(
    credit_card_cnts_sum[['SK_ID_CURR', 'AMT_BAL_DIV_MONTHS']], 
    on = 'SK_ID_CURR', how = 'inner')
credit_card_trim.shape

(103558, 11)

In [165]:
credit_card_trim.head()

,SK_ID_CURR,AMT_BALANCE,AMT_TOTAL_RECEIVABLE,AMT_RECIVABLE,AMT_RECEIVABLE_PRINCIPAL,AMT_INST_MIN_REGULARITY,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,MONTHS_BALANCE,CNT_DRAWINGS_POS_CURRENT,AMT_BAL_DIV_MONTHS
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,
0,100006,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-3.5,NaN,0.000000
1,100011,54482.111149,54433.179122,54433.179122,52402.088919,3956.221849,0.054054,0.054054,-38.5,0.0,1415.119770
2,100013,18159.919219,18101.079844,18101.079844,17255.559844,1454.539551,0.255556,0.239583,-48.5,0.0,374.431324
3,100021,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-10.0,NaN,0.000000
4,100023,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-7.5,NaN,0.000000


In [166]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in credit_card_trim.columns.levels[0]:
    # Skip the id name

    if (var != 'SK_ID_CURR') and (var != 'AMT_BAL_DIV_MONTHS'):
        # Iterate through the stat names
        stat = credit_card_trim.columns.levels[1][1]
        # Make a new column name for the variable and stat
        columns.append('%s_%s' % (var, stat))

# Assign the list of columns names as the dataframe column names
columns.append('AMT_BAL_DIV_MONTHS')
credit_card_trim.columns = columns

In [168]:
credit_card_trim.head()

,SK_ID_CURR,AMT_BALANCE_mean,AMT_INST_MIN_REGULARITY_mean,AMT_RECEIVABLE_PRINCIPAL_mean,AMT_RECIVABLE_mean,AMT_TOTAL_RECEIVABLE_mean,CNT_DRAWINGS_ATM_CURRENT_mean,CNT_DRAWINGS_CURRENT_mean,CNT_DRAWINGS_POS_CURRENT_mean,MONTHS_BALANCE_mean,AMT_BAL_DIV_MONTHS
0,100006,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-3.5,NaN,0.000000
1,100011,54482.111149,54433.179122,54433.179122,52402.088919,3956.221849,0.054054,0.054054,-38.5,0.0,1415.119770
2,100013,18159.919219,18101.079844,18101.079844,17255.559844,1454.539551,0.255556,0.239583,-48.5,0.0,374.431324
3,100021,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-10.0,NaN,0.000000
4,100023,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,-7.5,NaN,0.000000


In [169]:
credit_card_trim.to_csv(path + 'credit_card_trim.csv', index=False)

### Merge all together

In [170]:
# Merge main DFs with secondary DFs
# app_train, app_test
# prev_app_trim, bureau_trim, credit_card_trim

app_train.shape

(307511, 241)

In [171]:
app_test.shape

(48744, 240)

In [172]:
app_train_merged = app_train.merge(prev_app_trim, on = 'SK_ID_CURR', how = 'left')
app_train_merged = app_train_merged.merge(bureau_trim, on = 'SK_ID_CURR', how = 'left')
app_train_merged = app_train_merged.merge(credit_card_trim, on = 'SK_ID_CURR', how = 'left')
app_train_merged.shape

(307511, 266)

In [173]:
app_test_merged = app_test.merge(prev_app_trim, on = 'SK_ID_CURR', how = 'left')
app_test_merged = app_test_merged.merge(bureau_trim, on = 'SK_ID_CURR', how = 'left')
app_test_merged = app_test_merged.merge(credit_card_trim, on = 'SK_ID_CURR', how = 'left')
app_test_merged.shape

(48744, 265)

In [175]:
for col in app_train_merged:
    if 365243 in app_train_merged[col].values:
        print(col)

SK_ID_CURR
NAME_PRODUCT_TYPE_walk-in_mean


In [ ]:
# Replace the anomalous values with nan
app_train_merged['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [180]:
for col in app_test_merged:
    if 365243 in app_train_merged[col].values:
        print(col)

SK_ID_CURR


In [185]:
app_train_merged['TARGET'].head()

0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [184]:
app_test_merged.shape

(48744, 265)

In [181]:
app_train_merged.to_csv(path + 'app_train_merged.csv', index=False)

In [182]:
app_test_merged.to_csv(path + 'app_test_merged.csv', index=False)